<a href="https://colab.research.google.com/github/SteinwayThor/eeg-features-movies/blob/main/neural_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTS
import os
from google.colab import drive
import bz2
import _pickle as cPickle

In [2]:
drive.mount('/content/drive', force_remount=True)

# Load any compressed pickle file
def decompress_pickle(file):
  data = bz2.BZ2File(file, 'rb')
  data = cPickle.load(data)
  return data

Mounted at /content/drive


In [3]:
chapter_lengths = {'01': 8054,
 '02': 5524,
 '03': 9462,
 '04': 7207,
 '05': 8719,
 '06': 6096,
 '07': 7959,
 '08': 10235,
 '09': 6058,
 '10': 5312,
 '11': 8965,
 '12': 7983,
 '13': 7350,
 '14': 8577,
 '15': 10077,
 '16': 9084,
 '17': 12400,
 '18': 6768,
 '19': 9473,
 '20': 10194,
 '21': 9451,
 '22': 6749,
 '23': 5323,
 '24': 3870,
 '25': 11284,
 '26': 4671,
 '27': 6227,
 '28': 8296,
 '29': 12115,
 '30': 9339,
 '31': 5317}

In [4]:
# Load in neural data
# NOTE: Takes a while!
neural_data_folder = '/content/drive/MyDrive/Movie Data/e0018RI/2023-04-02_e0018RI_00'

neural_data_dir = os.listdir(neural_data_folder)

neural_data = {}
channels = []
for f in neural_data_dir:
  channel_name = f[f.rfind('_')+1:-5]
  channels.append(channel_name)
  channel_data = decompress_pickle(os.path.join(neural_data_folder, f))
  neural_data[channel_name] = channel_data
  print(f'Loaded in channel {channel_name}')

for f in neural_data_dir:
  channel_name = f[f.rfind('_')+1:-5]
  channels.append(channel_name)
  channel_data = decompress_pickle(os.path.join(neural_data_folder, f))
  neural_data[channel_name] = channel_data
  print(f'Loaded in channel {channel_name}')

print(channels)

Loaded in channel Events
Loaded in channel PULV15
Loaded in channel PULV14
Loaded in channel PULV13
Loaded in channel PULV12
Loaded in channel PULV11
Loaded in channel PULV10
Loaded in channel PULV9
Loaded in channel PULV8
Loaded in channel PULV7
Loaded in channel PULV6
Loaded in channel PULV5
Loaded in channel PULV4
Loaded in channel PULV3
Loaded in channel PULV2
Loaded in channel PULV1
Loaded in channel TPOCC10
Loaded in channel TPOCC9
Loaded in channel TPOCC8
Loaded in channel TPOCC7
Loaded in channel TPOCC6
Loaded in channel TPOCC5
Loaded in channel TPOCC4
Loaded in channel TPOCC3
Loaded in channel TPOCC2
Loaded in channel TPOCC1
Loaded in channel TOCC12
Loaded in channel TOCC11
Loaded in channel TOCC10
Loaded in channel TOCC9
Loaded in channel TOCC8
Loaded in channel TOCC7
Loaded in channel TOCC6
Loaded in channel TOCC5
Loaded in channel TOCC4
Loaded in channel TOCC3
Loaded in channel TOCC2
Loaded in channel TOCC1
Loaded in channel POCC10
Loaded in channel POCC9
Loaded in channel 

KeyboardInterrupt: ignored

In [ ]:
concatenated_